### todo
- "newtrial_frame", we might not need it in the future
- save name

In [2]:
import numpy as np
import pandas as pd
import scipy.io
from pathlib import Path
from kilosort.io import load_ops

date = 20250313

# Define Path and Load Files
results_dir = Path(r'D:\20250313\kilosort_bankAB_phy')
APtimestamps = np.load(r"D:\20250313\2025-03-13_15-09-41\Record Node 101\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeA-AP\timestamps.npy")
save_path = r'D:\20250313\zarya_20250313__bankAB_unit.mat'

spike_clusters = np.load(results_dir / 'spike_clusters.npy')
spike_times = np.load(results_dir / 'spike_times.npy')
channel_positions = np.load(results_dir / 'channel_positions.npy')
channel_map =  np.load(results_dir / 'channel_map.npy')
templates =  np.load(results_dir / 'templates.npy')

cluster_group = pd.read_csv(results_dir / 'cluster_info.tsv', sep='\t')
good_count = (cluster_group["group"] == "good").sum()
test = cluster_group["depth"][0]
print(f"Number of 'good' clusters: {good_count}")
print(test)

ModuleNotFoundError: No module named 'kilosort'

In [ ]:
chan_best = (templates**2).sum(axis=1).argmax(axis=-1)
chan_best = channel_map[chan_best]
template_amplitudes = ((templates**2).sum(axis=(-2,-1))**0.5)
uniq_spike_clusters = np.unique(spike_clusters)
spike_counts = np.unique(spike_clusters, return_counts=True)[1]
channel_y_positions = channel_positions[:, 1]
newtrial_frame = np.where(np.diff(APtimestamps) < -1) [0]
print(newtrial_frame)

In [16]:
uniq_spike_clusters

array([ 15,  16,  24,  27,  28,  30,  34,  37,  39,  40,  41,  44,  51,
        52,  55,  56,  73,  81,  85,  86,  87,  94,  96,  97,  99, 100,
       102, 104, 105, 110, 117, 119, 121, 122, 132, 133, 168, 171, 172,
       174, 207, 208, 209, 210, 211, 212, 213, 214, 217, 218, 219, 224,
       227, 231, 232, 233, 250, 262, 263, 272, 290, 292, 295, 299, 308,
       317, 321, 323, 326, 332, 334, 335, 336, 337, 338, 339, 340, 347,
       352, 353, 356, 357, 372, 383, 395, 406, 409, 414, 415, 417, 435,
       438, 439, 444, 447, 448, 449, 450, 451, 459, 460, 461, 462, 464,
       465, 466, 468, 469, 470, 471, 473, 476, 478, 482, 489, 494, 499,
       507, 508, 511, 512, 520, 522, 529, 531, 534, 536, 537, 540, 551,
       558, 562, 570, 575, 576, 578, 580, 582, 588, 591, 593, 596, 597,
       598, 609, 618, 622, 627, 631, 632, 635, 640, 641, 649, 655, 657,
       658, 670, 673, 676, 677, 682, 684, 687, 688, 691, 692, 695, 696,
       700, 702, 706, 714, 715, 719, 723, 724, 727, 728, 731, 73

In [8]:
## 20250313 version

depth = np.zeros(len(uniq_spike_clusters))
group = np.zeros(len(uniq_spike_clusters))
cluster_id = np.zeros(len(uniq_spike_clusters))
ch = np.zeros(len(uniq_spike_clusters))
spiketimes = {i: [] for i in range(len(uniq_spike_clusters))}


for i, cluster in enumerate(uniq_spike_clusters):
    idx = np.where(spike_clusters == cluster)
    unit_kilo_frame  = spike_times[idx]

    unit_spike_time = APtimestamps[unit_kilo_frame]
    
    cluster_id[i] = cluster
    spiketimes[i].extend(unit_spike_time)
    
    depth[i] = cluster_group["depth"][i]
    group[i] = int(cluster_group.loc[cluster_group["cluster_id"] == cluster, "group"].values[0] == "good")

spiketimes = np.array([spiketimes[i] for i in range(len(uniq_spike_clusters))], dtype=object)
units = {
    'depth': depth,
    'cluster_id': cluster_id,
    'groups': group,
    'spiketimes': spiketimes
}

scipy.io.savemat(save_path, {'units': units})

print(f"Data saved at '{save_path}'.")

Data saved at 'D:\20250313\zarya_20250313__bankAB_unit.mat'.


In [28]:
## 20241220 version

depth = np.zeros(len(uniq_spike_clusters))
group = np.zeros(len(uniq_spike_clusters))
cluster_id = np.zeros(len(uniq_spike_clusters))
ch = np.zeros(len(uniq_spike_clusters))
spiketimes = {i: [] for i in range(len(uniq_spike_clusters))}

for i, cluster in enumerate(uniq_spike_clusters):
    idx = spike_clusters == cluster
    unit_kilo_frame  = spike_times[idx]
    # trial_frame = (unit_kilo_frame >= newtrial_frame)
    # unit_kilo_frame = unit_kilo_frame[trial_frame]

    unit_spike_time = APtimestamps[unit_kilo_frame]
    
    cluster_id[i] = cluster
    spiketimes[i].extend(unit_spike_time)
    
    chbest = chan_best[i]
    ch[i] = chbest
    depth[i] = channel_y_positions[chbest]
    group[i] = 

spiketimes = np.array([spiketimes[i] for i in range(len(uniq_spike_clusters))], dtype=object)
units = {
    'depth': depth,
    'ch': ch,
    'cluster_id': cluster_id,
    'spiketimes': spiketimes
}

scipy.io.savemat(save_path, {'units': units})

print(f"Data saved at '{save_path}'.")

ValueError: operands could not be broadcast together with shapes (2117,) (0,) 